In [ ]:
###

### Inizio Terza parte

In [16]:
import json
from typing import List
from google.cloud import language
import pandas as pd
from google.cloud import language
from html.parser import HTMLParser

In [17]:
%run functions.ipynb

In [18]:
from google.cloud import storage
import pandas as pd
from io import BytesIO

# Your GCS bucket name
bucket_name = 'ccai-storage'

# Paths to the Parquet files in your GCS bucket
articles_parquet_path = 'pipeline/step3_pipeline.parquet'

# Initialize the GCS client
client = storage.Client()

# Load the Parquet files
final_df2 = load_parquet_from_gcs(bucket_name, articles_parquet_path)

In [19]:
print(final_df2['split'].value_counts())

split
TRAIN    10
Name: count, dtype: int64


In [20]:
final_df2

,uri,url,title,body,date,time,dateTime,dateTimePub,lang,isDuplicate,...,date_column,year,month,year_month,topic,split,shares_scaled,body_pre,score,magnitude
0,7384228286,https://www.avvenire.it/opinioni/pagine/le-pag...,Stella dell'assenza /10. Le pagine ancora non ...,Quella circulazion che sì concetta\n\npareva i...,2023-02-04,20:20:00,2023-02-04T20:20:00Z,2023-02-04T20:18:00Z,ita,False,...,2023-02-04,2023,2,2023-2,Arts and Entertainment,TRAIN,0.961538,circulazion sì concetta pareva te lume refless...,0.1,16.200001
1,7564762186,https://www.ilfattoquotidiano.it/2023/05/27/ce...,"Cent'anni di don Milani, il profeta che parla ...","Intervista a Mario Lancisi, tra i più esperti ...",2023-05-27,07:08:00,2023-05-27T07:08:00Z,2023-05-27T07:03:00Z,ita,False,...,2023-05-27,2023,5,2023-5,Arts and Entertainment,TRAIN,2.923077,"Intervista Mario Lancisi , esperti biografi Pr...",0.0,20.400000
2,7434699975,https://www.ilfattoquotidiano.it/2023/03/07/91...,"91 ritocchi al corpo in due, gli sposi entrano...","Nella scorsa puntata de Lo Show dei record, Ge...",2023-03-07,12:06:00,2023-03-07T12:06:00Z,2023-03-07T12:01:00Z,ita,False,...,2023-03-07,2023,3,2023-3,Arts and Entertainment,TRAIN,-0.153846,"scorsa puntata de Show record , Gerry Scotti c...",0.1,5.100000
3,7626351074,https://www.ilfattoquotidiano.it/2023/07/06/gi...,"Giovanni Allevi: ""Per superare il dolore, dedi...",Il pianista e compositore Giovanni Allevi ha t...,2023-07-06,14:54:00,2023-07-06T14:54:00Z,2023-07-06T14:51:00Z,ita,False,...,2023-07-06,2023,7,2023-7,Arts and Entertainment,TRAIN,3.500000,pianista compositore Giovanni Allevi trovato r...,0.1,3.400000
4,7406181074,https://www.corriere.it/spettacoli/23_febbraio...,L&#8217;eredit&#224; di Lisa Marie Presley &#4...,"Alla morte dell'unica figlia del mitico Elvis,...",2023-02-17,14:57:00,2023-02-17T14:57:00Z,2023-02-17T14:36:00Z,ita,False,...,2023-02-17,2023,2,2023-2,Arts and Entertainment,TRAIN,-0.038462,"morte dell'unica figlia mitico Elvis , vedova ...",-0.3,0.600000
5,7394819359,https://www.ilfattoquotidiano.it/2023/02/10/in...,Incidente di percorso per Lorella Cuccarini du...,Per la serata delle cover il cantante in gara ...,2023-02-10,20:37:00,2023-02-10T20:37:00Z,2023-02-10T20:34:00Z,ita,False,...,2023-02-10,2023,2,2023-2,Arts and Entertainment,TRAIN,-0.269231,serata cover cantante gara Olly deciso portare...,0.1,2.800000
6,7451376867,https://www.avvenire.it/rubriche/pagine/videog...,"Videogame con Carlo Acutis, guida nel metaver...","Mentre si continua a parlare di ""Prega.org"", l...",2023-03-17,06:51:00,2023-03-17T06:51:00Z,2023-03-17T06:51:00Z,ita,False,...,2023-03-17,2023,3,2023-3,Arts and Entertainment,TRAIN,-0.269231,"Mentre continuum parlare `` Prega.org '' , cha...",0.1,2.200000
7,2023-11-175599696,https://www.repubblica.it/spettacoli/cinema/20...,"'Palazzina Laf', Michele Riondino e la classe ...",Arriva nelle sale il film che racconta una vic...,2023-11-30,07:02:28,2023-11-30T07:02:28Z,2023-11-30T00:01:00Z,ita,False,...,2023-11-30,2023,11,2023-11,Arts and Entertainment,TRAIN,-0.269231,Arriva sale film racconta vicenda realmente ac...,0.1,3.900000
8,7448614148,https://www.corriere.it/moda/cards/bomber-mani...,È bomber mania: da Chiara Biasi a Emma Marrone...,Il capospalla numero uno di questo periodo è i...,2023-03-15,16:40:00,2023-03-15T16:40:00Z,2023-03-15T16:39:00Z,ita,False,...,2023-03-15,2023,3,2023-3,Arts and Entertainment,TRAIN,-0.307692,capospalla numero periodo bomber . Tutte star ...,0.1,5.100000
9,2023-11-166423184,https://www.corriere.it/spettacoli/cards/mrs-d...,"""Mrs. Doubtfire"": chi ha denunciato abusi, chi...",Il film diretto da Chris Columbus ha come prot...,2023-11-22,07:17:52,2023-11-22T07:17:52Z,2023-11-22T07:17:37Z,ita,False,...,2023-11-22,2023,11,2023-11,Arts and Entertainment,TRAIN,0.576923,film diretto Chris Columbus protagonista strao...,0.3,6.300000


### ----------------------------------------------------------------------------------------

### Analyze Moderation

In [21]:
def moderate_text(text: str) -> language.ModerateTextResponse:
    client = language.LanguageServiceClient()
    document = language.Document(
        content=text,
        type_=language.Document.Type.PLAIN_TEXT,
    )
    return client.moderate_text(document=document)

In [22]:
class MLStripper(HTMLParser):
    def __init__(self):
        super().__init__()
        self.reset()
        self.strict = False
        self.convert_charrefs = True
        self.text = []

    def handle_data(self, d):
        self.text.append(d)

    def get_data(self):
        return ''.join(self.text)

def strip_html_tags(html):
    s = MLStripper()
    s.feed(html)
    return s.get_data()

def moderate_text(text: str) -> language.ModerateTextResponse:
    client = language.LanguageServiceClient()
    document = language.Document(
        content=text,
        type_=language.Document.Type.PLAIN_TEXT,
    )
    
    return client.moderate_text(document=document)

def analyze_documents(df: pd.DataFrame, num_documents: int = 2, column:str = 'text'):
    documents = []
    for index, row in df.iterrows():
        linetext_cleaned = row.get(column, '')
        if linetext_cleaned:
            # Perform moderation analysis on the cleaned text
            response = moderate_text(linetext_cleaned)
            categories = []
            confidences = []
            for category in response.moderation_categories:
                categories.append(category.name)
                confidences.append(category.confidence)
            document = row.to_dict()
            for category, confidence in zip(categories, confidences):
                # Create new columns for each category with confidence scores
                document[category] = confidence
            documents.append(document)
            if len(documents) >= num_documents:
                break  # Stop after processing the specified number of documents

    df_result = pd.DataFrame(documents)
    return df_result


In [23]:
#row_count = len(final_df2)
row_count=10
# Example usage:
# df is your pandas DataFrame containing the documents
# num_documents is the number of documents to process
df_moderated = analyze_documents(final_df2, num_documents=row_count, column="body_pre")

In [24]:
df_moderated_v2 = df_moderated.copy()

In [25]:
df_moderated_v2

,uri,url,title,body,date,time,dateTime,dateTimePub,lang,isDuplicate,...,Violent,Firearms & Weapons,Public Safety,Health,Religion & Belief,Illicit Drugs,War & Conflict,Politics,Finance,Legal
0,7384228286,https://www.avvenire.it/opinioni/pagine/le-pag...,Stella dell'assenza /10. Le pagine ancora non ...,Quella circulazion che sì concetta\n\npareva i...,2023-02-04,20:20:00,2023-02-04T20:20:00Z,2023-02-04T20:18:00Z,ita,False,...,0.028125,0.000000,0.005008,0.023965,0.983158,0.009554,0.010417,0.008380,0.006837,0.002938
1,7564762186,https://www.ilfattoquotidiano.it/2023/05/27/ce...,"Cent'anni di don Milani, il profeta che parla ...","Intervista a Mario Lancisi, tra i più esperti ...",2023-05-27,07:08:00,2023-05-27T07:08:00Z,2023-05-27T07:03:00Z,ita,False,...,0.052846,0.018182,0.097938,0.006445,0.983158,0.009554,0.208955,0.922330,0.041096,0.435185
2,7434699975,https://www.ilfattoquotidiano.it/2023/03/07/91...,"91 ritocchi al corpo in due, gli sposi entrano...","Nella scorsa puntata de Lo Show dei record, Ge...",2023-03-07,12:06:00,2023-03-07T12:06:00Z,2023-03-07T12:01:00Z,ita,False,...,0.002514,0.000000,0.001439,0.034950,0.053885,0.009554,0.000477,0.000601,0.003163,0.002938
3,7626351074,https://www.ilfattoquotidiano.it/2023/07/06/gi...,"Giovanni Allevi: ""Per superare il dolore, dedi...",Il pianista e compositore Giovanni Allevi ha t...,2023-07-06,14:54:00,2023-07-06T14:54:00Z,2023-07-06T14:51:00Z,ita,False,...,0.111111,0.000000,0.010695,0.787313,0.981429,0.361111,0.058824,0.004959,0.007634,0.011976
4,7406181074,https://www.corriere.it/spettacoli/23_febbraio...,L&#8217;eredit&#224; di Lisa Marie Presley &#4...,"Alla morte dell'unica figlia del mitico Elvis,...",2023-02-17,14:57:00,2023-02-17T14:57:00Z,2023-02-17T14:36:00Z,ita,False,...,0.128049,0.000000,0.256757,0.034950,0.053885,0.057377,0.010417,0.011268,0.515901,0.920962
5,7394819359,https://www.ilfattoquotidiano.it/2023/02/10/in...,Incidente di percorso per Lorella Cuccarini du...,Per la serata delle cover il cantante in gara ...,2023-02-10,20:37:00,2023-02-10T20:37:00Z,2023-02-10T20:34:00Z,ita,False,...,0.028125,0.000000,0.010695,0.071066,0.006456,0.057377,0.010417,0.002819,0.006837,0.011976
6,7451376867,https://www.avvenire.it/rubriche/pagine/videog...,"Videogame con Carlo Acutis, guida nel metaver...","Mentre si continua a parlare di ""Prega.org"", l...",2023-03-17,06:51:00,2023-03-17T06:51:00Z,2023-03-17T06:51:00Z,ita,False,...,0.052846,0.018182,0.020568,0.096305,0.983158,0.038462,0.010417,0.030905,0.042169,0.002938
7,2023-11-175599696,https://www.repubblica.it/spettacoli/cinema/20...,"'Palazzina Laf', Michele Riondino e la classe ...",Arriva nelle sale il film che racconta una vic...,2023-11-30,07:02:28,2023-11-30T07:02:28Z,2023-11-30T00:01:00Z,ita,False,...,0.278261,0.000000,0.097938,0.005747,0.002467,0.029412,0.121951,0.019293,0.014898,0.176871
8,7448614148,https://www.corriere.it/moda/cards/bomber-mani...,È bomber mania: da Chiara Biasi a Emma Marrone...,Il capospalla numero uno di questo periodo è i...,2023-03-15,16:40:00,2023-03-15T16:40:00Z,2023-03-15T16:39:00Z,ita,False,...,0.002514,0.018182,0.097938,0.000181,0.002467,0.002336,0.867188,0.000601,0.003163,0.000000
9,2023-11-166423184,https://www.corriere.it/spettacoli/cards/mrs-d...,"""Mrs. Doubtfire"": chi ha denunciato abusi, chi...",Il film diretto da Chris Columbus ha come prot...,2023-11-22,07:17:52,2023-11-22T07:17:52Z,2023-11-22T07:17:37Z,ita,False,...,0.000000,0.000000,0.000000,0.000000,0.001459,0.000000,0.000000,0.000000,0.000000,0.000000


### ----------------------------------------------------------------------------------------
### clean the columns

In [23]:
# Remove '&' character from column names
df_moderated_v2.columns = df_moderated.columns.str.replace('&', '')
df_moderated_v2.columns = df_moderated_v2.columns.str.replace(r'\s+', ' ')
df_moderated_v2.columns = df_moderated_v2.columns.str.replace(',', '')
df_moderated_v2.columns = df_moderated_v2.columns.str.replace(' ', '_')

#df.columns = df.columns.str.replace(r'\s+', '')
df_moderated_v2.columns 

Index(['uri', 'url', 'title', 'body', 'date', 'time', 'dateTime',
       'dateTimePub', 'lang', 'isDuplicate', 'dataType', 'sentiment',
       'eventUri', 'relevance', 'image', 'authors', 'sharesFacebook',
       'sourceTitle', 'sourceLocationLabel', 'categoryLabels',
       'categoryWeights', 'importanceRank', 'alexaGlobalRank',
       'alexaCountryRank', 'date_column', 'year', 'month', 'year_month',
       'topic', 'split', 'shares_scaled', 'body_pre', 'score', 'magnitude',
       'num_documents', 'Toxic', 'Insult', 'Profanity', 'Derogatory', 'Sexual',
       'Death_Harm__Tragedy', 'Violent', 'Firearms__Weapons', 'Public_Safety',
       'Health', 'Religion__Belief', 'Illicit_Drugs', 'War__Conflict',
       'Politics', 'Finance', 'Legal'],
      dtype='object')

### ----------------------------------------------------------------------------------------
### Save to the bucket

In [24]:
### Salvo il df come parquet to the bucket
bucket_name="ccai-storage"

folder = "pipeline"
# Define your Google Cloud Storage bucket name and Parquet file name
parquet_file_name = 'final_df3.parquet'

# Save DataFrames to GCS as Parquet files in the 'fabio' folder
save_df_to_gcs_parquet(df_moderated_v2, bucket_name, folder, parquet_file_name)

## Fine terza parte